In [24]:
#!/usr/bin/env python

import argparse
import sys
import os
import pickle

from tqdm import tqdm
from joblib import Parallel, delayed

import jams
import pumpp

import h5py

In [17]:
def process_arguments(args):
    parser = argparse.ArgumentParser(description=__doc__)

    parser.add_argument('--sample-rate', dest='sr', type=float, default=44100.,
                        help='Sampling rate for audio analysis')

    parser.add_argument('--hop-length', dest='hop_length', type=int,
                        default=512,
                        help='Hop length for audio analysis')
    
    parser.add_argument('--nfft', dest='n_fft', type=int,
                        default=4096,
                        help='Number of FFT for STFT')

    parser.add_argument('--jobs', dest='n_jobs', type=int,
                        default=1,
                        help='Number of jobs to run in parallel')

    parser.add_argument('input_path', type=str,
                        help='Path for directory containing (audio_vocal, audio_instrument)')

    parser.add_argument('output_path', type=str,
                        help='Path to store pump output')

    return parser.parse_args(args)

In [18]:
def root(x):
    return os.path.splitext(os.path.basename(x))[0]

In [19]:
OUTPUT_PATH = '/Users/yuwang/projects/deepunet/'

In [20]:
def make_pump(sr, hop_length, n_fft):
    p_stft = pumpp.feature.STFTMag(name='stft',
                                     sr=sr, hop_length=hop_length, n_fft=n_fft,
                                     log=False, conv='tf')

    pump = pumpp.Pump(p_stft)

    # Save the pump
    with open(os.path.join(OUTPUT_PATH, 'pump.pkl'), 'wb') as fd:
        pickle.dump(pump, fd)

    return pump

In [26]:
def convert(aud_vocal, aud_inst, pump, outdir):
    data = {}
    data_vocal = pump.transform(aud_vocal)
    data_inst = pump.transform(aud_inst)
    data['stft/mag'] =  data_vocal['stft/mag'] + data_inst['stft/mag']
    data['output/mag'] =  data_inst['stft/mag']
    
    fname = os.path.extsep.join([os.path.join(outdir, root(aud_inst)), 'h5'])
    
    with h5py.File(fname, 'w') as hf:
        hf.update(**data)

In [22]:
output_path = '/Users/yuwang/projects/deepunet/pump/'

In [28]:
if __name__ == '__main__':
    pump = make_pump(22050, 512, 1024)
    
    AUDIO_VOCAL = jams.util.find_with_extension('/Users/yuwang/projects/deepunet/test/vocal/', 'mp3')
    AUDIO_INST = jams.util.find_with_extension('/Users/yuwang/projects/deepunet/test/instrumental/', 'mp3')

    # Make sure there are the same number of files
    assert len(AUDIO_VOCAL) == len(AUDIO_INST)
    # And that they're in agreement
    assert all([root(_1) == root(_2) for (_1, _2) in zip(AUDIO_VOCAL, AUDIO_INST)])
    
    Parallel(n_jobs=1)(delayed(convert)(aud_vocal, aud_inst, pump, output_path) for (aud_vocal, aud_inst) in zip(AUDIO_VOCAL, AUDIO_INST))